# BiomeFUN 
# Stoichiometric Modelling

## Basic Properties of a Netowrk
Blocked reactions on small model

In [86]:
import numpy as np
from scipy.linalg import null_space

rxns = np.array(['v1','v2','v3','v4'])

S = np.array([
    #v1  v2  v3  v4
    [1,  -1, -1,  0],   # A
    [0,   1, -1, -1],   # B
    [0,  -1,  0,  1],   # NADH
    [0,   1,  0, -1],   # NAD
    [0,   0,  1,  0],   # NADPH
    [0,   0, -1,  0],   # NADP
], dtype=float)

NS = null_space(S)            # Calculating the nullspace of S
blocked_mask = np.all(np.isclose(NS, 0.0, atol=1e-12), axis=1)
blocked = rxns[blocked_mask]  # Which reactions did not carry a solution in the NS?

# Print blocked reactions if they exist
for r in blocked:
    print(f"Reaction {r} is blocked")

Reaction v3 is blocked


Blocked reactions with less cofactors

In [89]:
rxns = np.array(['v1','v2','v3','v4'])

S = np.array([
    #v1  v2  v3  v4
    [1,  -1, -1,  0],   # A
    [0,   1,  1, -1],   # B
    [0,  -1, -1,  1],   # NADH
    [0,   1,  1, -1],   # NAD
], dtype=float)

NS = null_space(S)            # columns = basis vectors of ker(S)
blocked_mask = np.all(np.isclose(NS, 0.0, atol=1e-12), axis=1)
blocked = rxns[blocked_mask]

for r in blocked:
    print(f"Reaction {r} is blocked")

## Basic properties of a Network

Internal cycles

In [107]:
# Shut down v1 by removing its column
keep = [1, 2, 3]   # indices of v2, v3, v4
S_reduced = S[:, keep]

# Calculate the NullSpace of this reduced model
NS_reduced = null_space(S_reduced, rcond=1e-12)

if NS_reduced.shape[1] == 0:
    print("No internal cycle: null space is trivial after v1=0")
else:
    print("Internal cycle detected!")
    # map back to reactions
    cycle = dict(zip(rxns[1:], NS_reduced[:,0].round(3)))
    print("Cycle flux distribution:", cycle)

Internal cycle detected!
Cycle flux distribution: {'v2': -0.707, 'v3': 0.707, 'v4': 0.0}


## Using the null space to interpret metabolic models  

The null space of the stoichiometric matrix gives insight into two key phenomena:  

### Blocked reactions  
- **Definition:** reactions that *cannot* carry any steady-state flux because their substrates/products cannot be balanced in the network.  
- **Implication:** they are effectively “dead” under the current network and bounds.  
- **Possible fixes:**  
  - Add missing exchange/transport reactions for metabolites that have no supply/consumption.  
  - Adjust reversibility or check for missing links.  
- **Can you ignore them?** Yes — but they will never contribute to flux distributions until fixed.  

### Internal cycles (futile loops)  
- **Definition:** sets of reactions that can circulate flux internally while still satisfying steady-state (`S v = 0`), even without any external input.  
- **Implication:** they represent thermodynamically infeasible or unconstrained fluxes.  
- **Possible fixes:**  
  - Refine reversibility assumptions.  
  - Use thermodynamic or experimental constraints to rule them out.  
- **Can you ignore them?** Yes — but be aware that they allow non-identifiable solutions; loopless FBA or additional constraints are often used to prevent them.  